In [1]:
import pandas as pd
poly_data = pd.read_excel('resulting_dataset.xlsx')

In [2]:
poly_data_cols_medians = [a for a in poly_data.columns if 'median' in a]
poly_data_cols_vars = [a for a in poly_data.columns if 'variance' in a]

In [3]:
new_df = pd.read_csv('cols.csv')

In [22]:
filled_data = pd.read_excel('merged_result.xlsx')

In [4]:
bests = pd.read_excel('max_r2.xlsx')
best_chars = list(bests['Charactristic'])

In [59]:
# Ваши листы данных
all_cols = best_chars
parts_list = ['Hansen parameter delta-h: hydrogen bonding', 'Hansen parameter delta-d: dispersion component'] 

# Находим названия, которые есть в all_cols, но отсутствуют в parts_list
missing_names = set(all_cols) - set(parts_list)

# Преобразуем результат обратно в список, если это необходимо
missing_names_list = list(missing_names)


In [26]:
dt = filled_data[missing_names_list]

In [29]:
# Identify columns with the suffix '_value_variance' and names from missing_names_list
selected_columns = [col for col in poly_data.columns if any(name in col for name in missing_names_list) and col.endswith('_value_variance')]

# Extract the relevant subset of the DataFrame
subset_poly_data = poly_data[selected_columns]

In [32]:
mains = poly_data[['polymer_name', 'SMILES']]

In [33]:
result_df = pd.concat([mains, dt, subset_poly_data], axis=1)

## Загрузка датасета

In [2]:
import pandas as pd
result_df = pd.read_excel('RESULT.xlsx')

In [5]:
selected_cols = [col for col in result_df.columns if any(name in col for name in best_chars) and col.endswith('_value_median')]

In [88]:
selected_cols[0].replace('_value_median', '')

'Elongation at yield'

In [111]:
train_df = result_df[result_df['Specific volume_value_variance'].isna() == True][selected_cols]
test_df = result_df[result_df['Specific volume_value_variance'].isna() == False][selected_cols]

In [113]:
result_df[result_df['Specific volume_value_variance'].isna() == False]['Specific volume_value_variance']

0        12.574191
1         0.002383
2         0.003568
3         0.004569
4         0.000092
           ...    
18219     0.000000
18220     0.000000
18221     0.000000
18222     0.000000
18309     0.000000
Name: Specific volume_value_variance, Length: 1739, dtype: float64

In [82]:
X_train, y_train = train_df.drop(['Specific volume_value_median'], axis=1), train_df['Specific volume_value_median']
X_test, y_test = test_df.drop(['Specific volume_value_median'], axis=1), test_df['Specific volume_value_median']

In [7]:
import os
import numpy as np
from sklearn.linear_model import  LinearRegression, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

def normalized_mse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    var_y = np.var(y_true)
    nmse = mse / var_y
    return nmse

def mean_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    # Avoid division by zero
    non_zero_indices = y_true != 0
    y_true_non_zero = y_true[non_zero_indices]
    y_pred_non_zero = y_pred[non_zero_indices]
    
    # Calculate MPE
    mpe = abs(np.mean((y_true_non_zero - y_pred_non_zero) / y_true_non_zero) * 100)
    return mpe

def root_mean_squared_error(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

def custom_metric(row):
    if row['var'] > 0:
        return 1 if abs(row['y_pred'] - row['y_test']) < row['var'] else 0
    else:
        return None
    
def run_knn(X_train, X_test, y_train, y_test):
    model = KNeighborsRegressor(n_neighbors=5)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    coefs = None  # KNN Regressor doesn't have coefficients
    return y_pred, coefs

def run_lasso(X_train, X_test, y_train, y_test):
    model = Lasso()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    coefs = model.coef_
    return y_pred, coefs

def run_elastic_net(X_train, X_test, y_train, y_test):
    model = ElasticNet()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    coefs = model.coef_
    return y_pred, coefs

def run_decision_tree(X_train, X_test, y_train, y_test):
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    coefs = None  # Decision Tree doesn't have coefficients
    return y_pred, coefs

def run_bagging(X_train, X_test, y_train, y_test):
    model = BaggingRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    coefs = None  # Bagging doesn't have coefficients
    return y_pred, coefs

def run_adaboost(X_train, X_test, y_train, y_test):
    model = AdaBoostRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    coefs = None  # AdaBoost doesn't have coefficients
    return y_pred, coefs

def run_xgboost(X_train, X_test, y_train, y_test):
    model = XGBRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    coefs = None  # XGBoost doesn't have coefficients
    return y_pred, coefs

def run_svr(X_train, X_test, y_train, y_test):
    model = SVR()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    coefs = None  # SVR doesn't have coefficients
    return y_pred, coefs

def run_gradient_boosting(X_train, X_test, y_train, y_test):
    model = GradientBoostingRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    coefs = None  # GradientBoostingRegressor doesn't have coefficients
    return y_pred, coefs

def run_linear_regression(X_train, X_test, y_train, y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    coefs = model.coef_
    return y_pred, coefs

def run_random_forest(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    coefs = model.feature_importances_
    return y_pred, coefs


# List of models and their names
models = [run_knn, run_lasso, run_elastic_net, run_decision_tree, run_bagging, run_adaboost,
          run_xgboost, run_svr, run_gradient_boosting, run_linear_regression, run_random_forest]
model_names = ['KNeighborsRegressor', 'Lasso', 'Elastic Net', 'Decision Tree', 'Bagging', 'AdaBoost', 'XGBoost',
               'SVR', 'Gradient Boosting', 'Linear Regression', 'Random Forest']


# List of models and their names
models = [run_knn]
model_names = ['KNeighborsRegressor']

for model, model_name in zip(models, model_names):
    
    # Specify the directory and file name
    model_directory = f'check/{model_name}'
    
    # Create the directory if it doesn't exist
    os.makedirs(model_directory, exist_ok=True)
        
    mse_list = []
    r2_list = []
    rmse_list = []
    nmse_list = []
    mae_list = []
    mpe_list = []
    overall_list = []
    data_list = []
    name_list = []

    for q in range(len(selected_cols)):

        cleaned_column_name = selected_cols[q].replace('_value_median', '')
        
        # Split the data into training and testing sets
        train_df = result_df[result_df[f'{cleaned_column_name}_value_variance'].isna() == True][selected_cols]
        test_df = result_df[result_df[f'{cleaned_column_name}_value_variance'].isna() == False][selected_cols]
        
        X_train, y_train = train_df.drop([f'{cleaned_column_name}_value_median'], axis=1), train_df[f'{cleaned_column_name}_value_median']
        X_test, y_test = test_df.drop([f'{cleaned_column_name}_value_median'], axis=1), test_df[f'{cleaned_column_name}_value_median']
        
        
        # Check if there are samples available before training the model
        if X_train.shape[0] == 0:
            print(f"No samples available for training the model for column: {cleaned_column_name}")
            continue

        data_size = X_test.shape[0]
        
        # Train the model on the training data
        try:
            y_pred, coefs = model(X_train, X_test, y_train, y_test)
        except ValueError as e:
            print(f"Error training the model for column {cleaned_column_name}: {e}")
            continue

        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        rmse = root_mean_squared_error(y_test, y_pred)
        nmse = normalized_mse(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        mpe = mean_percentage_error(y_test, y_pred)

        indexes_to_show = y_test.index
        y_var = result_df[result_df[f'{cleaned_column_name}_value_variance'].isna() == False][f'{cleaned_column_name}_value_variance']
        tst = list(y_test)
        prd = list(y_pred)
        ndf = pd.DataFrame({'y_test': tst, 'y_pred': prd, 'var': y_var})

        # Apply the custom metric to each row
        ndf['metric'] = ndf.apply(custom_metric, axis=1)

        # Calculate the overall metric (excluding rows where var <= 0)
        filtered_df = ndf.dropna(subset=['metric'])
        overall = filtered_df['metric'].mean()

        data_list.append(data_size)
        name_list.append(cleaned_column_name)
        mse_list.append(mse)
        r2_list.append(r2)
        rmse_list.append(rmse)
        nmse_list.append(nmse)
        mae_list.append(mae)
        mpe_list.append(mpe)
        overall_list.append(overall)

    # Create a DataFrame for the current model
    res_model = pd.DataFrame({
        'Data Size': data_list,
        'Name': name_list,
        'MSE': mse_list,
        'R2': r2_list,
        'RMSE': rmse_list,
        'NMSE': nmse_list,
        'MAE': mae_list,
        'MPE': mpe_list,
        'Overall': overall_list
    })

    # Save the results to an Excel file
    with pd.ExcelWriter(f'{model_directory}/results_{model_name}.xlsx', engine='openpyxl', mode='w') as writer:
        res_model.to_excel(writer, sheet_name='Results', index=False)

No samples available for training the model for column: Volume resistivity


D:\Users\ivan\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.541e+05, tolerance: 2.864e+02
  model = cd_fast.enet_coordinate_descent(
D:\Users\ivan\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.301e+03, tolerance: 3.653e+00
  model = cd_fast.enet_coordinate_descent(


No samples available for training the model for column: Volume resistivity


D:\Users\ivan\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
D:\Users\ivan\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
D:\Users\ivan\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.123e+04, tolerance: 

No samples available for training the model for column: Volume resistivity


D:\Users\ivan\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
D:\Users\ivan\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.538e+07, tolerance: 1.623e+04
  model = cd_fast.enet_coordinate_descent(
D:\Users\ivan\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 

No samples available for training the model for column: Volume resistivity
No samples available for training the model for column: Volume resistivity
No samples available for training the model for column: Volume resistivity
No samples available for training the model for column: Volume resistivity
No samples available for training the model for column: Volume resistivity
No samples available for training the model for column: Volume resistivity
No samples available for training the model for column: Volume resistivity
No samples available for training the model for column: Volume resistivity


In [ ]:
import os
import pandas as pd

def parse_all_xlsx_files(folder_path):
    all_dfs = []

    # Walk through the directory and its subdirectories
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Check if the file has a .xlsx extension
            if file.endswith('.xlsx'):
                file_path = os.path.join(root, file)

                # Try to read the Excel file into a DataFrame
                try:
                    df = pd.read_excel(file_path)
                    
                    # Store both the file name and the DataFrame in a tuple
                    result = (file, df)
                    all_dfs.append(result)
                except Exception as e:
                    print(f"Error reading file {file}: {e}")

    return all_dfs

# Specify the folder path containing Excel files
folder_path = 'check2'

# Call the function to parse all Excel files in the folder and its subfolders
list_of_dfs = parse_all_xlsx_files(folder_path)

In [ ]:
import matplotlib.pyplot as plt

# Define a sequence of markers
markers = ['o', '^', 's', 'D', 'v', '<', '>', 'p', '*', 'h']

# Create an empty list to store legend handles and labels
legend_handles = []

# Assuming list_of_dfs is already populated
for d in range(len(list_of_dfs)):
    index_to_plot = d
    
    df_to_plot = list_of_dfs[index_to_plot][1]
    file_name = list_of_dfs[index_to_plot][0]

    # Split the file name by underscore and get the second part
    entity = file_name.split('_')[1]

    # Split the entity by dot and get the first part
    parsed_entity = entity.split('.')[0]

    # Set a range for the y-axis to avoid extreme values
    y_axis_lower_limit = -0.05  # Set your desired lower limit
    y_axis_upper_limit = 1.05   # Set your desired upper limit

    # Filter values within the specified range
    filtered_df = df_to_plot[(df_to_plot['Overall'] >= y_axis_lower_limit) & (df_to_plot['Overall'] <= y_axis_upper_limit)]

    # Set 'Name' column as the index
    filtered_df.set_index('Name', inplace=True)

    # Get the marker for the current plot
    current_marker = markers[d % len(markers)]

    # Create a scatter plot using 'Name' as x-axis labels and the current marker
    scatter = plt.scatter(filtered_df.index, filtered_df['Overall'], label=f'VM of {parsed_entity}', marker=current_marker)

    # Add zebra-like background
    for i, index_name in enumerate(filtered_df.index):
        color = 'lightgray' if i % 2 == 0 else 'white'
        # Adjust the span to cover the entire area between x-axis ticks
        plt.axvspan(i - 0.5, i + 0.5, facecolor=color, alpha=0.3, zorder=-1)

    # Append the scatter plot handle to the legend_handles list
    legend_handles.append(scatter)

# Draw horizontal red lines at y=0 and y=1
plt.axhline(0, color='red', linestyle='--', linewidth=1, label='y=0')
plt.axhline(1, color='red', linestyle='--', linewidth=1, label='y=1')

# Set labels and title
plt.title('Variance Metric (VM) score depending on the characteristic')
plt.xlabel('Name of characteristic')
plt.ylabel('Variance Metric score')

# Rotate the figure 90 degrees clockwise
plt.xticks(rotation=90)

# Add legend outside the plot
plt.legend(handles=legend_handles, loc='upper left', bbox_to_anchor=(1, 1))

# Set a white background
plt.gca().set_facecolor('white')

# Adjust figure size if needed
# Adjust figure size if needed
plt.gcf().set_size_inches(12, 8)

# Save the figure with an explicit bounding box
plt.savefig('VM_all_checked.eps', bbox_inches='tight', format='eps', dpi=300)
# Show the plot
plt.show()